# SCC-ICMC-USP - 1o. semestre de 2024

## SCC5871/MAI5025 - Exercício 2

## Profa. Roseli A. F. Romero


Nro do grupo: - 

Alunos:

1.   João Francisco Baiochi (n.ºUSP 7547674)
2.   
  
Notebook source: [GitHub](https://github.com/baiochi/SCC5871_Ver6_Turma3_2024)

---  

Para esse exercício, vamos utilizar o dataset Iris. Ele descreve atributos sobre 3 tipos de flores.
O objetivo é classificar qual o tipo de flor conforme os atributos disponíveis. Vamos trabalhar apenas com as duas primeiras classes para que
o problema de classificação binária.

In [1]:
import warnings
import numpy as np
from sklearn.datasets import load_iris
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)

iris = load_iris()
# Nessa primeira parte, vamos trabalhar apenas com as duas primeiras features e as duas primeiras classes
X = iris.data[:100, :4]
y = iris.target[:100]

---

### Questão 1.


- a) Utilizando o sklearn, defina uma [MLP para classificação binária](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) com a seguinte configuração:
    - função de ativação ReLU;
    - duas camadas escondidads com 10 neurônios cada;
    - taxa de aprendizado igual a 1e-2;
    - utilizando o algoritmo de otimização de gradiente descendente estocástico;
    - utilizando 10 iterações máximas (épocas);
    - use random_state=1234


- b) Treine a MLP definida no conjunto Iris simplificado definido na questçao anterior, e calcule a cross-entropy loss binária seguindo a definição a seguir.

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

# Configurando a instância MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10),
                    activation='relu',
                    learning_rate_init=1e-2,
                    solver='sgd',
                    max_iter=10,
                    random_state=1234)

# Treinando o modelo
mlp.fit(X, y)

# Fazendo as predições
y_pred = mlp.predict_proba(X)

# Cálculo da cross-entropy loss
loss = log_loss(y, y_pred)

print(f'Cross-entropy loss: {loss}')

Cross-entropy loss: 0.43160839201457435


---

### Questão 2.

Para avaliar os modelos que serão testados, implemente a função `evaluate_model()`. Essa função recebe um modelo de classificador genérico (`model`) e avalia sua acurácia utilizando **10-fold stratified cross-validation**, retornando a média das acurácias de cada fold. O parâmetro `X` indica os dados e `y` os labels.
- Sugestão: há duas formas de implementar a validação cruzada: treinar manualmente os modelos nos [splits gerados](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) ou utilizar a função [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) do sklearn. Atente-se ao cálculo da acurácia.

- Para garantir uma melhor performance dos algoritmos, faça o preprocessamento desses dados através da classe `sklearn.preprocessing.StandardScaler`.

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score


def evaluate_model(model, X, y):
    scores = cross_val_score(model, X, y,
                             cv=StratifiedKFold(n_splits=10),
                             scoring='accuracy')
    # print(scores)
    return np.mean(scores)


# Aplicando o preprocessamento dos dados
scaler = StandardScaler()
X_transf = scaler.fit_transform(X)

# Testando a função
print(f'Acurácia média: {evaluate_model(mlp, X_transf, y) * 100:.2f}%')

Acurácia média: 50.00%


---

### Questão 3.

Agora para estruturar e organizar melhor nossos testes, vamos utilizar as estruturas de dicionário do Python. Por exemplo, se formos definir dois modelos de Multi-Layer Perceptron, podemos escrever:

```
experimentos = {
    "MLP camada escondida (5,)": MLPClassifier(hidden_layer_sizes=(5,),
    "MLP camada escondida (5,5)": MLPClassifier(hidden_layer_sizes=(5,5)        
}
```

Isso pode ser feito pois o Python trata funções como funções de primeira classe. Isso é, funções podem ser tratadas como variáveis.

Portanto, defina um dicionário de experimentos com ao menos 3 modelos de MLP (`sklearn.neural_network.MLPClassifier`). Para isso varie parâmetros como o número de camadas escondidas, função de ativação e número de neurônios.

- Dica: Ver documentação em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
- Utilize um número de iterações >= 50 para garantir convergência.
- Experimente diferentes taxas de aprendizado e número máximo de iterações (épocas) de forma a garantir convergência no treino.

In [4]:
experimentos = {
    "MLP camada escondida (5,)": MLPClassifier(hidden_layer_sizes=(5,),
                                               activation='relu',
                                               learning_rate_init=1e-3,
                                               solver='sgd',
                                               max_iter=50,
                                               random_state=1234),
    "MLP camada escondida (10,5)": MLPClassifier(hidden_layer_sizes=(10, 5),
                                                 activation='logistic',
                                                 learning_rate_init=1e-2,
                                                 solver='lbfgs',
                                                 max_iter=100,
                                                 random_state=1234),
    "MLP camada escondida (15,10,5)": MLPClassifier(hidden_layer_sizes=(15, 10, 5),
                                                     activation='relu',
                                                     learning_rate_init=1e-5,
                                                     solver='sgd',
                                                     max_iter=150,
                                                     random_state=1234),
    "MLP camada escondida (20,15,10,5)": MLPClassifier(hidden_layer_sizes=(20, 15, 10, 5),
                                                        activation='relu',
                                                        learning_rate_init=1e-5,
                                                        solver='lbfgs',
                                                        max_iter=100,
                                                        random_state=1234)
}

---

### Questão 4.

- a) Para cada modelo instanciado na Questão 3, utilize a função criada na questão 3 para calcular sua acurácia. Exiba o nome do modelo e sua acurácia.
- b) Determine qual o melhor classificador dentre os especificados e justifique sua escolha.

In [5]:
from sklearn.model_selection import train_test_split

X = iris.data.copy()
y = iris.target.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Avaliando os modelos
for model_name, model in experimentos.items():
    acc = evaluate_model(model, X_train, y_train)
    print(f'{model_name}: acurácia = {acc * 100:.2f}%')

MLP camada escondida (5,): acurácia = 31.67%
MLP camada escondida (10,5): acurácia = 96.67%
MLP camada escondida (15,10,5): acurácia = 30.83%
MLP camada escondida (20,15,10,5): acurácia = 95.00%


O modelo com melhor desempenho foi o "MLP camada escondida (10,5)" com acurácia de 96.67%. Um diferencial foi a utilização do `solver='lbfgs'` que é um otimizador de segunda ordem, adequado para conjunto de dados pequenos. Aumentar o número de camadas e quantidades de neurônios não teve um grande impacto, visto que o último modelo possuiu mais parâmetros e obteve uma acurácia menor.  
Por fim precisamos fazer a comparação da acurácia entre os dados de treino e teste para verificar se o modelo está sofrendo de overfitting.

In [6]:
mlp = experimentos['MLP camada escondida (10,5)']
mlp.fit(X_train, y_train)

# Train accuracy
y_pred = mlp.predict(X_train)
acc = np.mean(y_pred == y_train)
print(f'Train accuracy: {acc * 100:.2f}%')

# Test accuracy
y_pred = mlp.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Test accuracy: {acc * 100:.2f}%')

Train accuracy: 99.17%
Test accuracy: 100.00%


Verificamos que o modelo obteve valores muito próximos de acurácia para os dados de treino e teste, indicando que não está sofrendo de overfitting e soube generalizar para dados não vistos anteriormente.